In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *

spark = SparkSession.builder.master('local[2]').getOrCreate()

In [22]:


distributionglobalsocialmediausersbyregion = spark.read.csv("hdfs://hdfs-nn:9000/user/Projeto TABD/Bronze/DistributionOfGlobalSocialMediaUsersByRegion.csv", 
                                                 header=True, 
                                                 sep=";")


In [23]:
distributionglobalsocialmediausersbyregion.printSchema()
distributionglobalsocialmediausersbyregion.show()
distributionglobalsocialmediausersbyregion.toPandas()

root
 |-- Distribution of global social media users 2020, by region: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)

+---------------------------------------------------------+----+----+
|Distribution of global social media users 2020, by region| _c1| _c2|
+---------------------------------------------------------+----+----+
|                                                     null|null|null|
|                                                     null|null|null|
|                                             Eastern Asia|29,6|in %|
|                                           Southeast Asia|13,4|in %|
|                                            Southern Asia|13,1|in %|
|                                         Northern America| 9,2|in %|
|                                            South America| 7,6|in %|
|                                     Central & Western...| 5,5|in %|
|                                             Western Asia| 4,4|i

,"Distribution of global social media users 2020, by region",_c1,_c2
0,None,None,None
1,None,None,None
2,Eastern Asia,"29,6",in %
3,Southeast Asia,"13,4",in %
4,Southern Asia,"13,1",in %
5,Northern America,"9,2",in %
6,South America,"7,6",in %
7,Central & Western Europe,"5,5",in %
8,Western Asia,"4,4",in %
9,Southern Europe,4,in %


In [24]:
distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.selectExpr("`Distribution of global social media users 2020, by region` as region", "`_c1` as percentage")
distributionglobalsocialmediausersbyregion.toPandas()

,region,percentage
0,None,None
1,None,None
2,Eastern Asia,"29,6"
3,Southeast Asia,"13,4"
4,Southern Asia,"13,1"
5,Northern America,"9,2"
6,South America,"7,6"
7,Central & Western Europe,"5,5"
8,Western Asia,"4,4"
9,Southern Europe,4


In [25]:
distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.filter((distributionglobalsocialmediausersbyregion.region != 'None'))

distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.withColumn(
    "percentage",
    f.regexp_replace(col("percentage"),",", "."))

In [26]:
distributionglobalsocialmediausersbyregion.toPandas()

,region,percentage
0,Eastern Asia,29.6
1,Southeast Asia,13.4
2,Southern Asia,13.1
3,Northern America,9.2
4,South America,7.6
5,Central & Western Europe,5.5
6,Western Asia,4.4
7,Southern Europe,4
8,Eastern Europe,3.3
9,Northern Africa,3


In [27]:
distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.withColumn(
    "percentage",
    distributionglobalsocialmediausersbyregion["percentage"].cast(FloatType()))

In [28]:
distributionglobalsocialmediausersbyregion.toPandas()

,region,percentage
0,Eastern Asia,29.6
1,Southeast Asia,13.4
2,Southern Asia,13.1
3,Northern America,9.2
4,South America,7.6
5,Central & Western Europe,5.5
6,Western Asia,4.4
7,Southern Europe,4.0
8,Eastern Europe,3.3
9,Northern Africa,3.0


In [29]:
distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.selectExpr("region as region", 
                                                                                                   "'0' as social_media_users",
                                                                                                   "'2020' as year",
                                                                                                   "'0' as mobile_penetration_percentage",
                                                                                                   "`percentage` as social_media_users_percentage", 
                                                                                                  )
distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.withColumn(
    "year",
    distributionglobalsocialmediausersbyregion["year"].cast(IntegerType()))

## Inserir o resto das colunas para ficar igual ao resto das tabelas 


## Trocar a ordem para ficar igual aos outros
distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion[['region', 'social_media_users', 'year', 'mobile_penetration_percentage', 'social_media_users_percentage']]

## Converter o tipo da coluna adicionada para ficar igual

distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.withColumn(
    "mobile_penetration_percentage",
    distributionglobalsocialmediausersbyregion["mobile_penetration_percentage"].cast(IntegerType()))

distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.withColumn(
    "social_media_users",
    distributionglobalsocialmediausersbyregion["social_media_users"].cast(DoubleType()))

distributionglobalsocialmediausersbyregion = distributionglobalsocialmediausersbyregion.withColumn(
    "social_media_users_percentage",
    distributionglobalsocialmediausersbyregion["social_media_users_percentage"].cast(FloatType()))

distributionglobalsocialmediausersbyregion.printSchema()
distributionglobalsocialmediausersbyregion.toPandas()

root
 |-- region: string (nullable = true)
 |-- social_media_users: double (nullable = true)
 |-- year: integer (nullable = true)
 |-- mobile_penetration_percentage: integer (nullable = true)
 |-- social_media_users_percentage: float (nullable = true)



,region,social_media_users,year,mobile_penetration_percentage,social_media_users_percentage
0,Eastern Asia,0.0,2020,0,29.6
1,Southeast Asia,0.0,2020,0,13.4
2,Southern Asia,0.0,2020,0,13.1
3,Northern America,0.0,2020,0,9.2
4,South America,0.0,2020,0,7.6
5,Central & Western Europe,0.0,2020,0,5.5
6,Western Asia,0.0,2020,0,4.4
7,Southern Europe,0.0,2020,0,4.0
8,Eastern Europe,0.0,2020,0,3.3
9,Northern Africa,0.0,2020,0,3.0


In [30]:
distributionglobalsocialmediausersbyregion \
    .write \
    .format("parquet") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000/warehouse/tabd.db/DistributionGlobalSocialMediaUsersByRegion2020/")